## 257. Introduction
- Go ahead and install PostgreSQL to get ready
- For windows users
    - Select your operating system
    - use 'Download the installer' option to download installer
- When you install, make sure that you enter valid username and password which you remember it (superuser: postgres, password: postgres)
    - Once the installation finishes, goto your commandline and you can use the following command
    ``` bash
    psql -U userName
    ```
    - ```psql``` is the PostgreSQL client or commandline tool, and then after above command, it'll prompt you for the password to emnter the command line using which we can create database, create users, create tables and perform CRUD operations

## 258. Setup the database
- You'll setup the ```employeedb``` Postgres database and ```employee``` table  inside it
- You'll then create a user 'test' with password 'password' and allow him to access our database and our table, so that we can use that user to connect to the database from within our node application
``` sql
    -- creates database
    CREATE DATABASE employeedb;
    -- uses database employeedb
    \c employeedb;
    -- creates table
    -- id column will be auto-incremented as it is a sequence
    CREATE TABLE employee (id SERIAL PRIMARY KEY, name VARCHAR(30), sal REAL);
    -- creates user(role)
    CREATE USER test WITH ENCRYPTED PASSWORD 'password';
    -- grant database level privileges to user 'test'
    GRANT ALL PRIVILEGES ON DATABASE employeedb TO test;
    -- grant table permission to user 'test'
    GRANT ALL PRIVILEGES ON TABLE employee TO test;
    -- grant permission of sequence created internally to user 'test'
    -- postgres internally creates a sequnce named 'employee_id_seq' , to which user 'test' should have
    -- otherwise it'll show error that 'test' user does not have access to the sequence rendering us
    -- unable to increment the value of 'id' column when you use the user 'test'
    GRANT ALL PRIVILEGES ON SEQUENCE employee_id_seq TO test;
    -- shows all records from employee table
    SELECT * FROM employee;

    -- DROP DATABASE employeedb
    -- DROP USER test
```
- currently, it does not show any records, but we'll add records from our python code

## 259. Install psycopg2
- To connect and work with Postgres databases from within your python program, we need the ```psycopg2``` package
- Just like we use ```MySQLCOnnector``` to connect to ```MySQL``` database, in case of ```Postgres```, we use ```Psycopg2``` package
- You can visit the PyPI wesbite for instaling Psycopg2 package using https://pypi.org/project/psycopg2/ and install the package using command ```pip install psycopg2``` or ```pip3 install psycopg2-binary```
- If you're using PyCharm, you need to ensure that the interpreter you're using in PyCharm also as access to this ```psycopg2``` package, because PyCharm installs its own interpreter
- Also, if you're using some other IDE, as long as you have performed the installation of such packages in the interpreter that your IDE is using, you should be good to go and able to import psycopg2 to work with it

## 260. Connect and Insert
- We'll programatically insert data into a postgres database table
- Before doing that execute this below command in ```psql``` shell
``` sql
SELECT * FROM employee;
```
- Use psycopg2 pacakge to establish a database connection and insert
- ```psycopg2.connect(dsn=None, connection_factory=None, cursor_factory=None, **kwargs)```
    - creates a new dataabse connection
    - connection parameters can be specified as a string:
        - ```psycopg2.connect("dbname=test user=postgres password=secret")```
    - or using a set of keyword arguments:
        - ```psycopg2.connect(database="test", user="postgres", password="secret")```
    - or as a mix of both. The basic connection parameters are
        - ```dbname``` : the database name
        - ```database``` : the database name (only as keyword argument)
        - ```user``` : user name used to authenticate
        - ```password``` : password used to authenticate
        - ```host``` : database host address (defaults to UNIX socket if not provided)
        - ```port``` : connection port number (defaults to 5432 if not provided)
    - Using the ```connection_factory``` parameter, a different class or connections factory can be specified. It should be a callable object taking a dsn argument
    - Using the ```cursor_factory``` parameter, a new default cursor factory will be used by the cursor()
    - Using ```async=True```, an asynchronous connection will be created, ```async_``` is a valid alias (for Python versions where ```async``` is a keyword)
    - Any other keyword parameter will be passed to the underlying client library, the list of supported parameters depends upon the library version
    - returns a connection object of type ```psycopg2.extensions.connection```
- ```psycopg2.extensions.connection.cursor(Name=None, cursor_factory=extensions.cursor, withhold=False)```
    - returns a new cursor of type ```psycopg2.extensions.cursor```
    - ```cursor_factory``` argument can be used to create non-standard cursors by passing a class different from the default. Note that the new class should be a sub-class of ```extensions.cursor```
- ```psycopg2.extensions.connection.commit()```
    - commit all changes to database
- ```psycopg2.extensions.connection.rollback()```
    - roll back all changes done to database
- ```psycopg2.extensions.connection.close()```
    - close the connection

In [2]:
# pginsert.py
import psycopg2
connection = psycopg2.connect(database='employeedb',
                              user='test',
                              password='password',
                              host='127.0.0.1',
                              port='5432')
print('Connected to employeedb')
# print("DEBUG:", connection) # debug
# print("DEBUG:", type(connection)) # debug
cursor = connection.cursor()
# print("DEBUG:", type(cursor)) # debug
cursor.execute("INSERT INTO employee (name, sal) VALUES ('Surya', 20000);")
# print("DEBUG:", connection) # debug
connection.commit()
# print("DEBUG:", cursor) # debug
# print("DEBUG:", connection) # debug
print("Employee Created")
cursor.close()
# print("DEBUG:", cursor) # debug
connection.close()
# print("DEBUG:", connection) # debug

Connected to employeedb
Employee Created


## 261. Fetch Data
- We'll fetch all the employees from the database
- ``` psycopg2.extensions.connection.cursor.fetchall()```
    - return all the remaining rows of the result set
    - returns a list of tuples
    - returns 'None' when no more data is available
    - rows are returned in the form of list of tuples (by default) or using the sequence factory previously set in the row factory attribute

In [4]:
# pgfetch.py
import psycopg2

connection = psycopg2.connect(database='employeedb',
                              user='test',
                              password='password',
                              host='127.0.0.1',
                              port='5432')
print("Connected to employeedb")
cursor = connection.cursor()
cursor.execute("SELECT * FROM employee;")
rows = cursor.fetchall()
for row in rows:
    print("ID:", row[0])
    print("Name:", row[1])
    print("Salary:", row[2])

cursor.close()
connection.close()

Connected to employeedb
ID: 2
Name: Surya
Salary: 20000.0


- Use UPDATE...SET query to update values in records

In [5]:
# pgupdate.py
import psycopg2

connection = psycopg2.connect(database='employeedb',
                              user='test',
                              password='password',
                              host='127.0.0.1',
                              port='5432')
print("Connected to employeedb")
cursor = connection.cursor()
cursor.execute("UPDATE employee SET sal=30000 WHERE id='2'")
# print("DEBUG: rows affected=", cursor.rowcount) # debug
connection.commit()
# connection.rollback()
print("Employee updated")
cursor.close()
connection.close()

Connected to employeedb
Employee updated


- Use DELETE query where you'll pass 'id' and delete the employee

In [9]:
# pgdelete.py
import psycopg2

connection = psycopg2.connect(database='employeedb',
                              user='test',
                              password='password',
                              host='127.0.0.1',
                              port='5432')
print("Connected to employeedb")
cursor = connection.cursor()
id = int(input("Enter id you want to delete: ")) # 2
cursor.execute("DELETE FROM employee WHERE id = '%d' " %(id))
# print("DEBUG: rows affected=", cursor.rowcount) # debug
connection.commit()
# connection.rollback()
print("Employee deleted with id : %d" %(id))
cursor.close()
connection.close()

Connected to employeedb
Enter id you want to delete: 2
Employee deleted with id : 2
